<a href="https://colab.research.google.com/github/wynniegross1/vegetation_anomalies/blob/main/vci_calculation_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Calculating VCI for a Specified Region of Interest**


---


Original GEE Script Author: Aaron Eubank

Python Script Adaptation: Wynnie Gross


---


*   This script is run primarily to get a map of Vegetation Condition Index values.
* This script defines a function to calculate this for a given region of interest.   
* The default dataset is MODIS 8-day product, however it is parameterizable to choose a different dataset.
*   Methodology adapted from GEARS - Geospatial Ecology and Remote Sensing - https://www.geospatialecology.com

In [7]:
import geemap
import ee
from google.colab import userdata
from datetime import datetime

# INITIALIZE GEE PROJECT #
ee.Authenticate()
ee.Initialize(project=userdata.get('projectname'))

def calculate_vci(
    region_of_interest,
    start_date,
    end_date,
    data_source='MODIS', # User can specify MODIS, Landsat, or Sentinel-2
    mask_cropland=True,
    cloud_masking=True
):
    """
    Calculates Vegetation Condition Index (VCI) for a given region and time period.

    Args:
        region_of_interest: ee.FeatureCollection. The area to analyze.
        start_date (str): Start date for the period (YYYY-MM-DD).
        end_date (str): End date for the period (YYYY-MM-DD).
        data_source (str): 'MODIS', 'Landsat', or 'Sentinel-2'.
        mask_cropland (bool): Whether to apply a cropland mask.
        cloud_masking (bool): Whether to apply cloud masking.

    Returns:
        tuple: (ee.Image, geemap.Map) The VCI image and the map object.
    """
    # Define data source parameters
    data_sources = {
        'MODIS': {
            'collection': 'MODIS/061/MOD09A1',
            'nir_band': 'sur_refl_b02',
            'red_band': 'sur_refl_b01',
            'cloud_band': 'StateQA',
            'cloud_mask_bit': 1 << 10,
            'scale': 500
        },
        'Landsat': {
            'collection': 'LANDSAT/LC08/C02/T1_L2',
            'nir_band': 'SR_B5',
            'red_band': 'SR_B4',
            'cloud_band': 'QA_PIXEL',
            'cloud_mask_bit': 1 << 3,
            'scale': 30
        },
        'Sentinel-2': {
            'collection': 'COPERNICUS/S2_SR_HARMONIZED',
            'nir_band': 'B8',
            'red_band': 'B4',
            'cloud_band': 'QA60',
            'cloud_mask_bit': 1 << 10,
            'scale': 10
        }
    }

    source = data_sources[data_source]

    # Helper functions
    def mask_clouds(image):
        """Masks clouds using the appropriate band for each data source."""
        QA = image.select([source['cloud_band']])
        return image.updateMask(QA.bitwiseAnd(source['cloud_mask_bit']).eq(0))

    def crop_mask(image):
        """Masks non-cropland areas using ESA WorldCover."""
        esa = ee.ImageCollection('ESA/WorldCover/v100')
        esa_latest = esa.limit(1, 'system:time_start').first()
        cropland = esa_latest.updateMask(
            esa_latest.select('Map').eq(40).clip(region_of_interest)
        )
        return image.updateMask(cropland)

    def add_ndvi(image):
        """Calculates NDVI and adds it as a band."""
        ndvi = image.normalizedDifference([source['nir_band'], source['red_band']]).rename('ndvi')
        return image.addBands(ndvi)

    # Load the image collection
    collection = ee.ImageCollection(source['collection'])

    # Filter by region of interest and date
    collection = collection.filterBounds(region_of_interest).filterDate(start_date, end_date)

    # Apply cloud masking if requested
    if cloud_masking:
        collection = collection.map(mask_clouds)

    # Calculate NDVI
    ndvi_collection = collection.map(add_ndvi).select('ndvi')

    # Apply cropland mask if requested
    if mask_cropland:
        ndvi_collection = ndvi_collection.map(crop_mask)

    # Calculate min and max NDVI
    ndvi_min = ndvi_collection.min()
    ndvi_max = ndvi_collection.max()

    # Calculate VCI
    vci = ndvi_collection.map(lambda img:
        img.subtract(ndvi_min).divide(ndvi_max.subtract(ndvi_min)).multiply(100)
    ).mean()

    # Create a map object
    Map = geemap.Map(center=region_of_interest.geometry().centroid().getInfo()['coordinates'][::-1], zoom=7)

    # Visualization parameters
    vci_vis = {
        'min': 0,
        'max': 100,
        'palette': ['red', 'yellow', 'green']
    }

    # Add the VCI layer to the map
    Map.addLayer(vci, vci_vis, 'Vegetation Condition Index')

    # Adding Outline to the Map
    outline = region_of_interest.style(fillColor='00000000')
    Map.addLayer(outline, {}, "Region of Interest")

    # Add legend
    Map.add_colorbar(
        vci_vis, label="Vegetation Condition Index",
        layer_name="VCI", orientation="horizontal"
    )

    return vci, Map

# Function to define region of interest based on country, admin1, and/or admin2
def get_region_of_interest(country, admin1_list=None, admin2_list=None):
    """
    Filters the admin2 boundaries dataset by country, and optionally by multiple admin1 and/or admin2 regions.

    Args:
        country (list): The name of the country (field 'admin0').
        admin1_list (list, optional): List of admin1 regions (field 'admin1').
        admin2_list (list, optional): List of admin2 regions (field 'admin2').

    Returns:
        ee.FeatureCollection: Filtered region of interest.
    """
    # Load the world admin2 boundaries dataset
    admin2_dataset = ee.FeatureCollection("projects/ee-aeubank/assets/world_admin2")

    # Start with a filter for the country
    filters = [ee.Filter.eq('admin0', country)]

    # Add filters for multiple admin1 regions if provided
    if admin1_list:
        admin1_filters = [ee.Filter.eq('admin1', region) for region in admin1_list]
        filters.append(ee.Filter.Or(*admin1_filters))

    # Add filters for multiple admin2 regions if provided
    if admin2_list:
        admin2_filters = [ee.Filter.eq('admin2', region) for region in admin2_list]
        filters.append(ee.Filter.Or(*admin2_filters))

    # Combine all filters using an AND filter
    combined_filter = ee.Filter.And(*filters)

    # Apply the combined filter to the dataset
    region_of_interest = admin2_dataset.filter(combined_filter)

    return region_of_interest

# Example usage
# Calculate VCI and get map
vci_result, vci_map = calculate_vci(
    region_of_interest = get_region_of_interest('Sudan', ['Sennar','Aj Jazirah']),
    start_date='2023-01-01',
    end_date='2023-12-31',
    data_source='Sentinel-2'  # Can be 'MODIS', 'Landsat', or 'Sentinel-2'
)

# Display the map
vci_map

Map(center=[12.89587933499483, 34.054257422974885], controls=(WidgetControl(options=['position', 'transparent_…